Part 1: Importing Necessary Libraries

In [31]:
import pandas as pd
import numpy as np
import random as rnd

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Deep learning libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
import webbrowser
from json.decoder import JSONDecodeError

# pytest
import pytest
import unittest
from unittest.mock import patch, Mock

Part 2: User Authentication
Reference: https://neokt.github.io/projects/audio-music-mood-classification/

In [32]:
# Create App With User Authentication

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="d62cb14d245c47668fd50431a942c046",
client_secret="fc28483fa9a74b22a90d076120fa3df8",
redirect_uri="http://localhost:5000/callback",
scope="user-library-read"))


results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], "-", track['name'])

0 Skryptonite - Polozhenie - Izzamuzzic Remix
1 104 - НЕ ЖАЛЬ (feat. Miyagi & Скриптонит)
2 Truwer - Весь в отца
3 Dequine - Deja Vu
4 Dose - Ветер
5 Автоспорт - Я не буду спать
6 Автоспорт - Пожалуйста, только не домой
7 Автоспорт - Песня о привязанности
8 King Harvest - Dancing in the Moonlight
9 Ashnikko - Daisy
10 Ashnikko - Slumber Party (feat. Princess Nokia)
11 Cuco - Si Me Voy (with The Marías)
12 Hozier - Work Song
13 Jordana - Better in the Dark
14 MUNA - Silk Chiffon
15 WILLIS - I Think I Like When It Rains
16 Ethel Cain - Crush
17 Taylor Swift - seven
18 Lizzy McAlpine - doomsday
19 Bon Iver - For Emma


Part 3: Understanding Spotipy and Querying
References: 
https://neokt.github.io/projects/audio-music-mood-classification/
https://spotipy.readthedocs.io/en/2.22.1/#api-reference

In [33]:
# get categories
categories = sp.categories(limit=50)
for i, cat in enumerate(categories['categories']['items']):
    print(i, cat['name'])

0 Top Lists
1 Hip-Hop
2 Pop
3 Country
4 Latin
5 Rock
6 Summer
7 Workout
8 R&B
9 Dance/Electronic
10 Netflix
11 Indie
12 Mood
13 Sleep
14 Christian & Gospel
15 Regional Mexican
16 Wellness
17 Chill
18 EQUAL
19 Gaming
20 Frequency
21 Kids & Family
22 Party
23 Decades
24 Fresh Finds
25 Jazz
26 Focus
27 Romance
28 Folk & Acoustic
29 K-Pop
30 Instrumental
31 Ambient
32 Alternative
33 In the car
34 Classical
35 Soul
36 Spotify Singles
37 Cooking & Dining
38 Punk
39 Pop culture
40 Blues
41 Desi
42 Arab
43 RADAR
44 Student
45 Anime
46 Tastemakers
47 Afro
48 Comedy
49 Metal


In [34]:
# get playlists from list of categories
cat = categories['categories']['items'][0]
playlists = sp.category_playlists(cat['id'], limit=50)
for i, playlist in enumerate(playlists['playlists']['items']):
    print(i, playlist['name'])

0 Today’s Top Hits
1 RapCaviar
2 Rock This
3 mint
4 Hot Country
5 Viva Latino
6 RNB X
7 Top 50 - USA
8 Top 50 - Global
9 Viral 50 - Global
10 Viral 50 - USA
11 New Music Friday


In [35]:
# get song ids from list of playlist ids
song_ids = []
for i, playlist in enumerate(playlists['playlists']['items']):
    playlist = sp.playlist(playlist['id'])
    for j, item in enumerate(playlist['tracks']['items']):
        song = item['track']
        song_ids.append(song['id'])
        
song_ids[:5]

['7gaA3wERFkFkgivjwbSvkG',
 '4xhsWYTOGcal8zt0J161CU',
 '3rUGC1vUpkDG9CZFHMur1t',
 '2IGMVunIBsBLtEQyoI1Mu7',
 '1BxfuPKGuaTgP7aM0Bbdwr']

In [36]:
# Use song ids to query audio features
features = []
for i in range(0,len(song_ids),50):
    audio_features = sp.audio_features(song_ids[i:i+50])
    for track in audio_features:
        features.append(track)

# Turn the features into a dataframe
features_df = pd.DataFrame(features)

features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.788,0.758,1,-6.513,1,0.0505,0.1980,0.000085,0.1040,0.823,119.008,audio_features,7gaA3wERFkFkgivjwbSvkG,spotify:track:7gaA3wERFkFkgivjwbSvkG,https://api.spotify.com/v1/tracks/7gaA3wERFkFk...,https://api.spotify.com/v1/audio-analysis/7gaA...,214994,4
1,0.943,0.558,2,-4.911,1,0.0568,0.0026,0.000002,0.0937,0.606,104.983,audio_features,4xhsWYTOGcal8zt0J161CU,spotify:track:4xhsWYTOGcal8zt0J161CU,https://api.spotify.com/v1/tracks/4xhsWYTOGcal...,https://api.spotify.com/v1/audio-analysis/4xhs...,138411,4
2,0.750,0.733,6,-3.180,0,0.0319,0.2560,0.000000,0.1140,0.844,111.018,audio_features,3rUGC1vUpkDG9CZFHMur1t,spotify:track:3rUGC1vUpkDG9CZFHMur1t,https://api.spotify.com/v1/tracks/3rUGC1vUpkDG...,https://api.spotify.com/v1/audio-analysis/3rUG...,131872,1
3,0.868,0.538,5,-8.603,1,0.1740,0.2690,0.000003,0.0901,0.732,99.968,audio_features,2IGMVunIBsBLtEQyoI1Mu7,spotify:track:2IGMVunIBsBLtEQyoI1Mu7,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,https://api.spotify.com/v1/audio-analysis/2IGM...,231750,4
4,0.552,0.702,9,-5.707,1,0.1570,0.1170,0.000021,0.1050,0.564,169.994,audio_features,1BxfuPKGuaTgP7aM0Bbdwr,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://api.spotify.com/v1/audio-analysis/1Bxf...,178427,4


Part 4: Creating moods and retrieving mood playlists
Moods obtained from: https://github.com/neokt/audio-music-mood-classification/blob/master/05-production.ipynb

In [37]:
# create a list of moods to classify playlists
moods = ['Peaceful', 'Easygoing', 'Tender', 'Romantic', 'Upbeat', 'Empowering', 'Lively', 'Excited', 'Stirring', 'Rowdy', 'Sentimental', 'Sophisticated','Sensual', 'Fiery', 'Energizing','Melancholy', 'Cool', 'Somber', 'Gritty','Yearning', 'Serious','Urgent', 'Defiant', 'Brooding', 'Aggressive']

In [39]:
# get Spotify playlists for each mood. Limit is 50 playlists per mood. For each mood, store the playlist name, id, and mood in a list of dictionaries
playlists = []
for mood in moods:
    mood_playlists = sp.search(q=mood, type='playlist', limit=600)
    for playlist in mood_playlists['playlists']['items']:
        playlists.append({'name': playlist['name'], 'id': playlist['id'], 'mood': mood})

# create a dataframe from the list of dictionaries
playlists_df = pd.DataFrame(playlists)

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Peaceful', 'limit': 600, 'offset': 0, 'type': 'playlist', 'market': None} returned 400 due to Invalid limit


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/search?q=Peaceful&limit=600&offset=0&type=playlist:
 Invalid limit, reason: None

In [ ]:
# get song ids for all the playlists for future querying
song_ids = []
for i, playlist in enumerate(playlists_df['id']):
    playlist = sp.playlist(playlist)
    for j, item in enumerate(playlist['tracks']['items']):
        song = item['track']
        song_ids.append(song['id'])
        
song_ids[:5]

['5hFkGfx038V0LhqI0Uff2J',
 '3RI7rnK1YgmyRcXq9pkMqQ',
 '6fjZ39b5dT9uZzFsVRLwSi',
 '1b1A6Z7pI8myEX3TSyweOk',
 '3xZBHTUT1gLAmDbPz77ETo']

In [ ]:
# create a dataframe from the list of dictionaries
playlists_df = pd.DataFrame(playlists)

playlists_df.head()

,name,id,mood
0,Peaceful Piano,37i9dQZF1DX4sWSpwq3LiO,Peaceful
1,Peaceful Meditation,37i9dQZF1DWZqd5JICZI0u,Peaceful
2,Peaceful Music (Instrumental),7MuGX2icynmq9zuklPCbnX,Peaceful
3,Peaceful music<3,37O3TVPbewdKcI2GBx0FbA,Peaceful
4,Peaceful Music,4Lw6TUrPoiQkTjXE46J8cD,Peaceful


In [ ]:
# add columns to playlists_df that contain the audio features for the songs
features = []

valid_song_ids = [song_id for song_id in song_ids if song_id is not None]

for i in range(0, len(valid_song_ids), 50):
    audio_features = sp.audio_features(valid_song_ids[i:i + 50])
    valid_audio_features = [track for track in audio_features if track is not None]
    features.extend(valid_audio_features)
    
# features[:5]
        
features_df = pd.DataFrame(features)

# merge the playlists_df and features_df dataframes
df = pd.merge(playlists_df, features_df, left_index=True, right_index=True)

df.head()


,name,id_x,mood,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id_y,uri,track_href,analysis_url,duration_ms,time_signature
0,Peaceful Piano,37i9dQZF1DX4sWSpwq3LiO,Peaceful,0.384,0.0189,1,-31.119,1,0.0395,0.992,...,0.0967,0.0834,62.479,audio_features,5hFkGfx038V0LhqI0Uff2J,spotify:track:5hFkGfx038V0LhqI0Uff2J,https://api.spotify.com/v1/tracks/5hFkGfx038V0...,https://api.spotify.com/v1/audio-analysis/5hFk...,130166,4
1,Peaceful Meditation,37i9dQZF1DWZqd5JICZI0u,Peaceful,0.435,0.0404,4,-26.262,1,0.0358,0.984,...,0.0952,0.1930,95.929,audio_features,3RI7rnK1YgmyRcXq9pkMqQ,spotify:track:3RI7rnK1YgmyRcXq9pkMqQ,https://api.spotify.com/v1/tracks/3RI7rnK1Ygmy...,https://api.spotify.com/v1/audio-analysis/3RI7...,168499,4
2,Peaceful Music (Instrumental),7MuGX2icynmq9zuklPCbnX,Peaceful,0.461,0.0286,0,-25.117,1,0.0482,0.995,...,0.1160,0.4710,113.851,audio_features,6fjZ39b5dT9uZzFsVRLwSi,spotify:track:6fjZ39b5dT9uZzFsVRLwSi,https://api.spotify.com/v1/tracks/6fjZ39b5dT9u...,https://api.spotify.com/v1/audio-analysis/6fjZ...,140103,4
3,Peaceful music<3,37O3TVPbewdKcI2GBx0FbA,Peaceful,0.386,0.0128,1,-28.660,1,0.0396,0.991,...,0.1090,0.1890,126.374,audio_features,1b1A6Z7pI8myEX3TSyweOk,spotify:track:1b1A6Z7pI8myEX3TSyweOk,https://api.spotify.com/v1/tracks/1b1A6Z7pI8my...,https://api.spotify.com/v1/audio-analysis/1b1A...,130134,4
4,Peaceful Music,4Lw6TUrPoiQkTjXE46J8cD,Peaceful,0.470,0.0243,5,-24.242,1,0.0354,0.994,...,0.1210,0.3310,81.805,audio_features,3xZBHTUT1gLAmDbPz77ETo,spotify:track:3xZBHTUT1gLAmDbPz77ETo,https://api.spotify.com/v1/tracks/3xZBHTUT1gLA...,https://api.spotify.com/v1/audio-analysis/3xZB...,154311,3


In [ ]:
# rename id_x and id_y to proper names
df.rename(columns={'id_x': 'playlist_id', 'id_y': 'song_id'}, inplace=True)
df.head()

,name,playlist_id,mood,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,song_id,uri,track_href,analysis_url,duration_ms,time_signature
0,Peaceful Piano,37i9dQZF1DX4sWSpwq3LiO,Peaceful,0.384,0.0189,1,-31.119,1,0.0395,0.992,...,0.0967,0.0834,62.479,audio_features,5hFkGfx038V0LhqI0Uff2J,spotify:track:5hFkGfx038V0LhqI0Uff2J,https://api.spotify.com/v1/tracks/5hFkGfx038V0...,https://api.spotify.com/v1/audio-analysis/5hFk...,130166,4
1,Peaceful Meditation,37i9dQZF1DWZqd5JICZI0u,Peaceful,0.435,0.0404,4,-26.262,1,0.0358,0.984,...,0.0952,0.1930,95.929,audio_features,3RI7rnK1YgmyRcXq9pkMqQ,spotify:track:3RI7rnK1YgmyRcXq9pkMqQ,https://api.spotify.com/v1/tracks/3RI7rnK1Ygmy...,https://api.spotify.com/v1/audio-analysis/3RI7...,168499,4
2,Peaceful Music (Instrumental),7MuGX2icynmq9zuklPCbnX,Peaceful,0.461,0.0286,0,-25.117,1,0.0482,0.995,...,0.1160,0.4710,113.851,audio_features,6fjZ39b5dT9uZzFsVRLwSi,spotify:track:6fjZ39b5dT9uZzFsVRLwSi,https://api.spotify.com/v1/tracks/6fjZ39b5dT9u...,https://api.spotify.com/v1/audio-analysis/6fjZ...,140103,4
3,Peaceful music<3,37O3TVPbewdKcI2GBx0FbA,Peaceful,0.386,0.0128,1,-28.660,1,0.0396,0.991,...,0.1090,0.1890,126.374,audio_features,1b1A6Z7pI8myEX3TSyweOk,spotify:track:1b1A6Z7pI8myEX3TSyweOk,https://api.spotify.com/v1/tracks/1b1A6Z7pI8my...,https://api.spotify.com/v1/audio-analysis/1b1A...,130134,4
4,Peaceful Music,4Lw6TUrPoiQkTjXE46J8cD,Peaceful,0.470,0.0243,5,-24.242,1,0.0354,0.994,...,0.1210,0.3310,81.805,audio_features,3xZBHTUT1gLAmDbPz77ETo,spotify:track:3xZBHTUT1gLAmDbPz77ETo,https://api.spotify.com/v1/tracks/3xZBHTUT1gLA...,https://api.spotify.com/v1/audio-analysis/3xZB...,154311,3


Data Preprocessing

In [ ]:
# Remove rows with missing values
df.dropna(inplace=True)

# Normalize the audio features with MinMaxScaler to get values between 0 and 1
scaler = MinMaxScaler()
df[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']] = scaler.fit_transform(df[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']])

# Encode the moods with LabelEncoder, and add an extra column to the dataframe with the encoded values
le = LabelEncoder()
df['mood_encoded'] = le.fit_transform(df['mood'])

df.head()

,name,playlist_id,mood,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,type,song_id,uri,track_href,analysis_url,duration_ms,time_signature,mood_encoded
0,Peaceful Piano,37i9dQZF1DX4sWSpwq3LiO,Peaceful,0.554284,0.173446,0.090909,0.640725,1.0,0.211896,0.978836,...,0.069963,0.089270,audio_features,5hFkGfx038V0LhqI0Uff2J,spotify:track:5hFkGfx038V0LhqI0Uff2J,https://api.spotify.com/v1/tracks/5hFkGfx038V0...,https://api.spotify.com/v1/audio-analysis/5hFk...,0.326611,0.75,12
1,Peaceful Meditation,37i9dQZF1DWZqd5JICZI0u,Peaceful,0.642733,0.372895,0.363636,0.839359,1.0,0.143123,0.936508,...,0.218856,0.363224,audio_features,3RI7rnK1YgmyRcXq9pkMqQ,spotify:track:3RI7rnK1YgmyRcXq9pkMqQ,https://api.spotify.com/v1/tracks/3RI7rnK1Ygmy...,https://api.spotify.com/v1/audio-analysis/3RI7...,0.643981,0.75,12
2,Peaceful Music (Instrumental),7MuGX2icynmq9zuklPCbnX,Peaceful,0.687825,0.263430,0.000000,0.886185,1.0,0.373606,0.994709,...,0.596522,0.510004,audio_features,6fjZ39b5dT9uZzFsVRLwSi,spotify:track:6fjZ39b5dT9uZzFsVRLwSi,https://api.spotify.com/v1/tracks/6fjZ39b5dT9u...,https://api.spotify.com/v1/audio-analysis/6fjZ...,0.408882,0.75,12
3,Peaceful music<3,37O3TVPbewdKcI2GBx0FbA,Peaceful,0.557752,0.116859,0.090909,0.741289,1.0,0.213755,0.973545,...,0.213422,0.612567,audio_features,1b1A6Z7pI8myEX3TSyweOk,spotify:track:1b1A6Z7pI8myEX3TSyweOk,https://api.spotify.com/v1/tracks/1b1A6Z7pI8my...,https://api.spotify.com/v1/audio-analysis/1b1A...,0.326346,0.75,12
4,Peaceful Music,4Lw6TUrPoiQkTjXE46J8cD,Peaceful,0.703434,0.223541,0.454545,0.921970,1.0,0.135688,0.989418,...,0.406331,0.247549,audio_features,3xZBHTUT1gLAmDbPz77ETo,spotify:track:3xZBHTUT1gLAmDbPz77ETo,https://api.spotify.com/v1/tracks/3xZBHTUT1gLA...,https://api.spotify.com/v1/audio-analysis/3xZB...,0.526514,0.50,12


In [ ]:
# Split data into train and test set
X = df[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']]
y = df['mood_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# save the dataset to a csv file
df.to_csv('spotify_moods.csv', index=False)